In [1]:
import copy
import pylab
import random
import math
import pandas as pd
import numpy as np

In [ ]:
['AA','AB','AC','AD','AE','AF','AG','AH','AI','AJ']

In [42]:
task_num = 10   ##设置任务数量
cyclenum = 3  ##设置周期数，这里与原代码中读取的周期数区分开
###只需要调整以上两个数就可以做实验
a = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
for i in range(26):
    for j in range(26):
        b = a[i]+a[j]
        a.append(b)
useclos = 'B:'+a[task_num]##得到工作表位置
excel_route="D:\\PCsoftware\\cplex\\xiaosuizhuanyong\\flexible_task13\\"+str(task_num)+"-"+str(cyclenum)+".xls" ##得到文件路径
useclos

'B:K'

In [44]:
excel_route="D:\\PCsoftware\\cplex\\xiaosuizhuanyong\\flexible_task13\\"+str(task_num)+"-"+str(cyclenum)+".xls"

'D:\\PCsoftware\\cplex\\xiaosuizhuanyong\\flexible_task13\\10-3.xls'

In [2]:
df1 = pd.read_excel(r'D:\PCsoftware\cplex\xiaosuizhuanyong\flexible_task13\13-2-607-1024.xls',sheet_name="Sheet1",usecols="B:Q")
df1

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,0,0,0,0,0,0,0,145,16,163,13,62,238,194.0,0.0
1,960,960,960,960,960,960,960,385,256,403,253,302,478,434.0,960.0
2,0,5,5,5,5,5,5,179,120,304,182,76,170,228.0,0.0
3,1,1,1,2,2,2,1,2,1,2,1,2,1,NaN,NaN
4,10000,139,16,394,71,94,70,319,65,203,13,49,273,26.0,0.0
5,10000,10000,127,283,237,707,459,35,157,140,118,252,360,312.0,89.0
6,10000,323,10000,467,183,149,71,17,37,46,25,15,160,86.0,347.0
7,10000,341,929,10000,169,19,337,134,29,92,28,144,7,207.0,261.0
8,10000,447,321,329,10000,731,269,156,73,318,346,223,31,101.0,222.0
9,10000,195,109,95,63,10000,709,6,44,42,277,126,10,13.0,49.0


In [3]:
# df1=np.array(df1)
# df1[3]

In [45]:
class ALNS():
    def __init__(self):
        """初始化参数"""
        self.T=100000 #定义初始温度
        self.q=0.9 #定义退火速率
        self.maxiter=100 #最大外循环次数
        self.maxl=30 #最大内循环次数
        self.wd=[0.5,0.5] #定义算子权重
        self.sd=[0,0] #定义算子分数
        self.ud=[0,0] #记录算子的使用次数
        """定义问题参数"""
        self.T1=[]
        self.T2=[]
        self.severtime=[]
        self.cycle=[]
        self.t=[]
        self.trace=[] #记录迭代曲线
        self.best=0 #记录最优解
        self.best_solution=[] #记录最优方案
        self.excel_name="Sheet1" #数据在excel文件的那个sheet中
        self.excel_route=r"D:\PCsoftware\cplex\xiaosuizhuanyong\flexible_task13\13-2-607-1024.xls" #excle的路径
        self.excel_col="B:Q" #定义数据在sheet的列范围
        self.empty=6 #空箱任务数量
        self.k=5 #车辆总数量
        self.Tmax=[660,960,1440]
        self.iter = 0  # 记录外循环次数
    
    def run(self):
        """算法主框架"""
        self.read_date() #提取数据
        self.divid_cycle() #将任务按周期划分
        s0=self.code() #初始化解    #s0 = solution[周期1[任务数1，任务数2],周期2[任务数]||||周期1[任务序号打乱]，周期2[任务序号打乱]]
        f0=self.trans_code(s0) #初始化解码，返回目标函数值
        self.best=f0 #初始化最优解
        self.best_solution=s0 #初始化最优方案
        self.trace.append(f0)
        self.pre_move()
        #self.iter=0  #记录外循环次数
        while self.iter<=self.maxiter:
            self.l=0
            while self.l<=self.maxiter:
                if f0<self.trace[-1]:
                    self.trace.append(f0)
                else:
                    self.trace.append(self.trace[-1])
                index=self.choice(self.wd) #根据轮盘度选择合适的算子，返回次数h-1
                if index==0:
                    s1=self.change_zero(s0) #使用第0号算子搜索，增减车辆数量，改变任务顺序
                else:
                    s1=self.change_one(s0)  #改变中间一部分任务顺序
                s1=self.move(s1)
                f1=self.trans_code(s1) #计算新解函数值
                """记录最优解"""
                if f1<self.best:
                    self.best=f1
                    self.best_solution=s1.copy()
                """"""
                self.ud[index]+=1 #更新算子使用次数
                if f1<f0:
                    s0=s1 #更新方案
                    f0=f1 #更新解
                    self.sd[index]+=1.2 #更新分数
                else:
                    if random.random()<=math.exp((f0-f1)/self.T):
                        s0 = s1  # 更新方案
                        f0 = f1  # 更新解
                        self.sd[index] += 0.8  # 更新分数
                    else:
                        self.sd[index] += 0.6
                self.wd[index]=0.5*self.wd[index]+0.5*self.sd[index]/self.ud[index] #更新权重
                self.l+=1
            self.iter+=1
            self.T=self.T*self.q #更新温度
#         self.output() #输出结果
        return [self.best,self.best_solution,self.trace]
        
    def read_date(self):
        """读取数据"""
        date=pd.read_excel(self.excel_route,sheet_name=self.excel_name,usecols =self.excel_col)
        date=np.array(date)
        self.T1=date[0]
        self.T2=date[1]
        self.severtime=date[2]##任务服务时间
        self.cycle=date[3][0:-2]   ###任务周期序号[1,1,2,2,1,2,1]
        self.t=date[4:]###任务节点行驶时间
        
    def code(self):###生成初始解
        """编码"""
        solution=[]
        self.cycle_num=int(max(self.cycle)) #周期数量
        """第一层编码，长度为车辆数*周期数"""                                 
        for i in range(self.cycle_num):                                     #self.c [周期1[任务序号1，任务序号2],周期2[任务序号1，任务序号2]，周期3[任务序号1，任务序号2]]
            list1=[random.randint(0,len(self.c[i])) for j in range(self.k)] #list1 每一辆车被分配的任务数量[车辆1[周期i任务数量内随机数？],车辆2[周期i任务序号随机数],车辆3[]]
            solution.append(list1)                                          #solution[周期1[车辆1数，车辆2数],周期2[车辆1，车辆2],周期3[车辆1，车辆2]]
        """第二层编码，长度为任务数量，每个周期任务的排列组合"""
        list2=copy.deepcopy(self.c)
        for i in range(int(max(self.cycle))):
            random.shuffle(list2[i])                                       
        for i in list2:
            solution.append(i)                                              #solution[周期1[任务数1，任务数2],周期2[任务数]||||周期1[任务序号打乱]，周期2[任务序号打乱]]
        return solution
    
    def trans_code(self,solution0,p=0): #输出车辆任务序列，时间窗惩罚，                               #solution0就是上面的solution
        """输入一个solution,解码"""
        solution=copy.deepcopy(solution0)
        f=0 #记录函数值
        time = [[0 for j in range(self.cycle_num)] for i in range(self.k)]  # 记录每一个车辆在3个周期下的作业时间
                                                                        #time[集卡1[周期1时间，周期2时间，周期3时间]，集卡2[周期1，周期2，周期3]，集卡3[]]
        for i in range(self.cycle_num): #遍历每一个周期i
            list1=solution[i+self.cycle_num] #记录每一个周期的任务     #list1周期i所有任务序号
            list2=[]
            for j in solution[i]:                                      #j=周期i车辆1、车辆2…分配任务数量
                if j==0: #表示车辆没有·被选中
                    continue
                elif list1 == []:
                    break
                else:
                    for z in range(len(solution[i])):   #周期i车辆数量，第z辆车
                        if solution[i][z]==j and z not in list2:  #周期i第z辆车任务数辆=j，并且任务数量不在list2。即任务全部由车辆z完成
                            vehic=z
                            list2.append(vehic) #存储周期i任务数量
                            break
                    if len(list1)<=j: #如果总任务数量少于该车辆被分配的任务数量
                        f1=self.cal_fit(0,list1) #计算函数值，按照周期1所有任务，按照list1顺序完成计算时间。即该车辆完成周期所有任务。
                        if p!=0:
                            print(f"周期{i},第{vehic}辆车的任务为：{list1}")
                        list1.clear() #将已完成的任务删除
                    else:
                        f1=self.cal_fit(0,list1[:j])#该车只能完成j个任务
                        if p!=0:
                            print(f"周期{i},第{vehic}辆车的任务为：{list1[:j]}")
                        list1=list1[j:]#前j个任务被完成，后面的任务由后面的车依次完成
                        #solution[vehic]=0
                    time[vehic][i]=f1   #time记录第v辆车第i周期的时间，下面判断司机时间窗约束
                    f+=f1
            for j in range(len(time)): #周期下时间窗约束 
                t3 = 0
                for k in range(self.cycle_num):
                    t1 = time[j][k]
                    if k<self.cycle_num-1:
                        t2 = time[j][k]+time[j][k+1]
                    t3 += time[j][k]
                    if t1>= self.Tmax[0]:
                        f=f*1.5 
                    elif t2 >= self.Tmax[1]:
                        f = f * 1.5
                    if t3>=self.Tmax[2]:
                        f = f * 1.5
            if list1!=[]:
                f+=500
        return f #反回函数值
    
    def cal_fit(self,a,list1):#计算目标函数a2（时间）
        """a,车辆开始作业的时间，list1为任务的列表"""
        a2=0
        if self.iter==self.maxiter+1:
            print(list1)
        for i in range(len(list1)):      #（由三个部分组成，开始场站到第一个任务，中间任务，最后一个任务到场站，这里只判断是否满足任务时间窗约束）
            if i==0:
                a+=self.t[0][list1[i]]   #初始场站到第一个点的时间
                a += self.severtime[0]   #加上服务时间
     
                a2+=self.t[0][list1[i]]   #a2是目标值（不包含服务时间），a是所有时间
                if self.iter==self.maxiter+1:
                    print('第一条路径：',self.t[0][list1[i]],0,list1[i])#打印路径和时间
            else:
                a += self.t[list1[i-1]][list1[i]]   #加上第一点到第二点、第二点到第三点……
                a += self.severtime[list1[i-1]]
                a2 += self.t[list1[i-1]][list1[i]]
                if self.iter==self.maxiter+1:
                    print(self.t[list1[i-1]][list1[i]],list1[i-1],list1[i])
                    
                if a<=int(self.T1[list1[i]]):  #任务左时间窗，发生等待，等待下一个任务开始
                    a=int(self.T1[list1[i]])
                    if self.iter == self.maxiter + 1:
                        print("满足任务的时间窗约束")
                if  a>int(self.T2[list1[i]]): #如果不满足每个点的时间窗约束
                    if self.iter == self.maxiter + 1:
                        print("不满足任务的时间窗约束")
                    # list1[i-1],list1[i]    
                    a2=a2*20
            a += self.t[list1[-1]][-1]  #加上最后一个任务到场站的时间
            a2 += self.t[list1[-1]][-1]
            if self.iter == self.maxiter + 1:
                print("满足任务的时间窗约束")
                print('最后一条路径',self.t[list1[-1]][-1], list1[- 1], "end")#打印路径
        return a2

    def divid_cycle(self):#生成self.c，每个周期的任务序号，序号从1开始，1，2，3……
        cycle_num=int(max(self.cycle)) #周期数
#         print(cycle_num)
#         print(int(cycle_num))
        self.c=[[] for i in range(cycle_num)] #对不同任务按照周期划分   [周期1[任务序号1，任务序号2],周期2[任务序号1，任务序号2]，周期3[任务序号1，任务序号2]]
#         print(cycle_num,self.cycle)
        for i in range(len(self.cycle)):
            self.c[int(self.cycle[i])-1].append(i+1)
    
    def choice(self,list1):
        """轮盘赌选择,返回子代列表"""
        list2 = [] # list2用来保存概率累加
        s=sum(list1)
        list1=[i/s for i in list1]
        for i in range(len(list1)):
            if i==0:
                list2.append(list1[0])
            else:
                list2.append(list2[-1]+list1[i])  # 计算累加概率
        for i in range(0,1):  # 表示轮盘选择次数
            h = 0  # h用来标记父代
            k = random.uniform(0, 1)  # 生成0，1的随即数
            for j in list2:  # 判断k在选择了那个父代
                h += 1
                if j >= k:
                    return h-1
                
    def change_zero(self,s):##增减车辆任务数量、改变任务顺序
        """搜索算子0"""
        s0=copy.deepcopy(s)
        for i in range(len(s0)):#周期数两倍
            if i<self.cycle_num:
                for j in range(len(s0[i])):#周期i车辆数。增加或减少车辆任务数量,概率各50%
                    r=random.uniform(0,1)
                    if r<=0.5:
                        #s0[i][j]周期i第j辆车任务数，len(self.c[i])第i周期总任务数
                        #self.c [周期1[任务序号1，任务序号2],周期2[任务序号1，任务序号2]，周期3[任务序号1，任务序号2]]
                        s0[i][j]+=int(round((len(self.c[i])-s0[i][j])*r*math.sqrt(1-(self.iter/self.maxiter)**2),0))
                    else:
                        s0[i][j]-=int(round((s0[i][j]-0) * r * math.sqrt(1 - (self.iter / self.maxiter) ** 2),0))
            else:
                for j in s0[i]:                #s0[i]周期i任务序列
                    if random.random()<=0.3:
                        x1=j    #周期i-cycle_num 每一个任务序号
                        index1=s0[i].index(j)
                        index2=random.randint(0,len(s0[i])-1)
                        x2=s0[i][index2]
                        s0[i].pop(index1)
                        s0[i].insert(index1,x2)
                        s0[i].pop(index2)
                        s0[i].insert(index2, x1)##一定概率改变任务顺序
        return s0#返回solution
    
    def change_one(self,s0):  ##打乱一部分顺序
        """搜索算子1"""
        new_s=[]
        for i in s0:
            if random.random()<=0.8:
                index1=random.randint(0,len(i)-1)
                index2=random.randint(0,len(i)-1)
                while index1==index2:
                    index2 = random.randint(0, len(i)-1)
                [index1,index2]=[min(index1,index2),max(index1,index2)]
                x1=i[:index1]
                x2=i[index1:index2]
                random.shuffle(x2)##中间一部分任务打乱顺序
                x3=i[index2:]
                x1.extend(x2)
                x1.extend(x3)
                new_s.append(x1.copy())
            else:
                new_s.append(i.copy())
        return new_s#返回任务顺序
    
    def pre_move(self):##柔性空箱周期序号，若不是柔性空箱则为[]
        """计算每个空箱任务可能存在的周期"""
        self.con_em=[[] for i in range(self.empty)]  #[[],[周期1，周期0]，[周期2，周期1]]空箱数量个
        for i in range(len(self.c)):  #[周期1[任务序号1，任务序号2],周期2[任务序号1，任务序号2]，周期3[任务序号1，任务序号2]]
            for j in self.c[i]:#周期i任务序号
                if j<=self.empty and i!=0:  #不是第一个周期，且序号小于空箱数量
                    self.con_em[j-1].append(i)
                    self.con_em[j-1].append(i-1)
    
    def move(self,s):##执行柔性空箱提前
        """非第一周期的空箱可以提前到上一个周期"""
        for i in range(len(s)):
            if i>=self.cycle_num:
                for j in s[i]:   #i-cycle_num周期任务序列，j是任务序号
                    if j<=self.empty and self.con_em[j-1]!=[]:#第j个任务是柔性空箱
                        if random.random()<=0.3:    #百分之三十概率提前
                            if i-self.cycle_num == self.con_em[j-1][0]:#空箱的原周期是该周期
                                s[self.con_em[j-1][1]+self.cycle_num].append(j)#提前周期
                                s[i].remove(j)
                            else:#原周期
                                s[self.con_em[j-1][0]+self.cycle_num].append(j)
                                s[i].remove(j)
#         print(s)
#         list1=[]
#         for i in s[3:]:
#             for j in i:
#                 if j in list1:
#                     print(s,11111)
#                 else:
#                     list1.append(j)
        return s
    
    
    
    
    def feasible solution(self,a,list1,fe_list,unf_list):#区分可行解，不可行解
        """a,车辆开始作业的时间，list1为任务的列表"""
        for i in range(len(list1)):      
            if i==0:
                a+=self.t[0][list1[i]]   #初始场站到第一个点的时间
                a += self.severtime[0]   #加上服务时间
            else:
                a += self.t[list1[i-1]][list1[i]]   #加上第一点到第二点、第二点到第三点……
                a += self.severtime[list1[i-1]]
                if a<=int(self.T1[list1[i]]):  #任务左时间窗，发生等待，等待下一个任务开始
                    a=int(self.T1[list1[i]])
                if  a>int(self.T2[list1[i]]): #如果不满足每个点的时间窗约束
                    unf_list.append(list1[:i-1])#不可行部分
                    fe_list.append(list1[i:])#可行部分
        return fe_list,unf_list
    
    
    
    
#     pylab .show()
    def output1(self,best,best_solution,trace):
        """输出结果与画图"""
        print(f"最优解为{best}")
        self.trans_code(best_solution,1)
        print('Best: ',best_solution)
        pylab.plot([i for i in range(len(trace))],trace)
        pylab.show()


In [46]:
if __name__=="__main__":
    
    instance=ALNS()
    instance.run()

NameError: name 'pd' is not defined

In [ ]:
# instance=ALNS()
# list1=[]
# list2=[]
# list3=[]
# for i in range(10):
#     best,solution,trace=instance.run()
#     list1.append(best)
#     list2.append(solution.copy())
#     list3.append(trace.copy())
# miny=min(list1)
# index=list1.index(miny)
# best_solution=list2[index]
# best_trace=list3[index]
# instance.output1(miny,best_solution,best_trace)